In [1]:
#%% IMPORTS
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
from scipy.stats import normaltest
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from IPython.display import display, Markdown, Latex
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.ensemble import ExtraTreesRegressor

## Methods

In [2]:
#Eliminate Outliers based on the interquantile
#datFrame: Data frame where the outliers will be eliminated.
#columnName: the name of the column where the outliers will be identified.
def eliminateOutliers (dataFrame, columnName):
    Q1 = dataFrame[columnName].quantile(0.25)
    Q3 = dataFrame[columnName].quantile(0.75)
    IQR = Q3 - Q1
    print('Initial dataframe size: '+str(dataFrame.shape))
    dataFrame = dataFrame[(dataFrame[columnName] < (Q3 + 1.5 * IQR)) & (dataFrame[columnName] > (Q1 - 1.5 * IQR))]
    print('Final dataframe size: '+str(dataFrame.shape))
    return dataFrame

In [3]:
# Create the boxplot graphs for the categorical variables
# dataFrame: Data frame associated to the property of interest (dfAirVoids, dfMS, dfMF, dfITS, dfTSR)
# propertyOfInterest: the name of the column where the property of interest is located.
# columnName1...4: The categorical columns to evaluate.
def displayBoxPlotGraphs (dataFrame, propertyOfInterest, columnName1, columnName2, columnName3, columnName4):
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,10))
    sns.boxplot(y = propertyOfInterest, x = columnName1, data=dataFrame,  orient='v' , ax=ax1)
    sns.boxplot(y = propertyOfInterest, x = columnName2, data=dataFrame,  orient='v' , ax=ax2)
    sns.boxplot(y = propertyOfInterest, x= columnName3, data=dataFrame,  orient='v' , ax=ax3)
    sns.boxplot(y= propertyOfInterest, x= columnName4, data=dataFrame,  orient='v' , ax=ax4)

##  Data Import 

In [5]:
#%%DATA READING AND INITIAL PREPROCESSING
numericColumns = ['Aggregate absorption [%]',
                  'Apparent specific gravity',
                    0.075,
                    0.3,
                    0.6,
                    2.36,
                    4.75,
                    9.5,
                    12.5,
                    19,
                    'Plastic particle size (mm)',
                    'Mixing speed (RPM)',
                    'Mixing Temperature',
                    'Mixing Time (hours)',
                    'Plastic Addition by bitumen weight (%)',
                    'Bitumen content in the sample'
                    ]
categoricalColumns = ['Modified asphalt Mix?',
                      'Agreggate Type',
                    'Aggregate absorption [%]',
                    'Filler used',
                    'Consolidated bitumen penetration grade',
                    'New Plastic Type',
                    'Plastic pretreatment',
                    'Plastic shape',
                    'Plastic Size',
                    'Mixing Process',
                    'Plastic melted previous to addition?',
                    'Aggregates replacement ?',
                    'Bitumen replacement?',
                    'Filler replacement',
                    'Property',
                    'Units']
#It returns the dataframe of interes based on the property - 'AirVoids', 'MS', 'MF', 'ITS', 'TSR'
def returnDf (propertyOfInterest):
    df = pd.read_excel('fileML.xlsx', sheet_name = propertyOfInterest, engine='openpyxl')
    df = df.set_index(propertyOfInterest + ' ID')
    df.loc[:,:'Units'] = df.loc[:,:'Units'].applymap(str)
    df.loc[:,:'Units'] = df.loc[:,:'Units'] .applymap(str.strip)
    df.replace('NS', np.nan, inplace = True)
    df[numericColumns] = df[numericColumns].replace('N/a', 0).astype(float)
    return df

In [6]:
dfMS = returnDf('MS')

## 1 Data Exploration
###  1.1 Total Sample

In [7]:
dfMS = eliminateOutliers(dfMS, 'MS of the sample (kN)')

Initial dataframe size: (406, 35)
Final dataframe size: (402, 35)


In [8]:
dfMS.iloc[:,2:].describe(include = 'all')

,Modified asphalt Mix?,Agreggate Type,Aggregate absorption [%],Apparent specific gravity,0.075,0.3,0.6,2.36,4.75,9.5,12.5,19,Filler used,Bitumen Type Penetration Grade,Consolidated bitumen penetration grade,New Plastic Type,Plastic pretreatment,Plastic shape,Plastic Size,Plastic particle size (mm),Mixing Process,Plastic melted previous to addition?,Mixing speed (RPM),Mixing Temperature,Mixing Time (hours),Aggregates replacement ?,Bitumen replacement?,Filler replacement,Plastic Addition by bitumen weight (%),Property,Units,Bitumen content in the sample,MS of the sample (kN)
count,402,262,242.000000,84.000000,325.000000,372.000000,344.000000,355.000000,372.000000,344.000000,357.000000,372.000000,161,402,402,377,402,402,320,307.000000,402,402,371.000000,385.000000,372.000000,402,402,402,400.000000,402,402,399.000000,402.000000
unique,2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,3,10,2,4,3,NaN,3,3,NaN,NaN,NaN,3,3,3,NaN,1,1,NaN,NaN
top,Yes,Granite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stone dust,60/70,50/70,PE,Physical,Shredded,Fine,NaN,Dry,No,NaN,NaN,NaN,No,No,No,NaN,MS,kN,NaN,NaN
freq,319,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81,141,276,150,319,289,148,NaN,200,240,NaN,NaN,NaN,300,285,317,NaN,402,402,NaN,NaN
mean,NaN,NaN,1.240269,2.686607,6.469785,15.079839,20.374273,37.026789,48.849785,68.082384,80.702129,92.721613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.332902,NaN,NaN,568.463612,42.883117,0.110914,NaN,NaN,NaN,19.153970,NaN,NaN,5.198835,14.458749
std,NaN,NaN,0.946592,0.078388,5.324256,5.749322,6.530935,7.607279,8.780095,10.803423,11.585460,7.199423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.836038,NaN,NaN,1272.097676,72.981514,0.282826,NaN,NaN,NaN,44.387176,NaN,NaN,0.820112,4.621909
min,NaN,NaN,0.130000,2.590000,2.190000,5.300000,9.000000,19.770000,23.950000,47.000000,66.630000,83.110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,2.500000,1.367840
25%,NaN,NaN,0.470000,2.600000,4.620000,10.660000,14.770000,33.470000,41.540000,59.850000,69.380000,84.460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,1.000000,NaN,NaN,4.800000,10.927500
50%,NaN,NaN,1.020000,2.685000,5.280000,13.780000,20.560000,35.860000,49.350000,67.840000,79.860000,94.940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.360000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,5.000000,NaN,NaN,5.200000,14.696960
75%,NaN,NaN,2.150000,2.725000,6.240000,17.670000,22.820000,39.000000,54.770000,75.970000,90.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,0.000000,150.000000,0.000000,NaN,NaN,NaN,12.000000,NaN,NaN,5.500000,17.287500
